In [1]:
# ============================================================
# ✅ MODEL PICKLE CONFIGURATION (CHANGE THIS CELL ONLY)
# ============================================================
MODEL_PKL = "/kaggle/input/network-snapshot-0002097-0-100-logarithmic-2/pytorch/default/1/network-snapshot-0002097-0.100_logarithmic_noiseinject.pkl"

# Example alternatives for other schedules:
# MODEL_PKL = "/kaggle/input/network-snapshot-last-linear-unconditional/..."
# MODEL_PKL = "/kaggle/input/network-snapshot-last-cosine-unconditional/..."
# MODEL_PKL = "/kaggle/input/network-snapshot-last-logrithmic/..."
# MODEL_PKL = "/kaggle/input/network-snapshot-last-quadratic-unconditional/..."


In [2]:
# ============================================================
# ✅ SETUP
# ============================================================
!git clone https://github.com/NVlabs/edm2.git /kaggle/working/edm2
%cd /kaggle/working/edm2
!pip install click tqdm psutil scipy pillow --quiet


Cloning into '/kaggle/working/edm2'...
remote: Enumerating objects: 60, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 60 (delta 13), reused 10 (delta 10), pack-reused 33 (from 1)
Receiving objects: 100% (60/60), 1.27 MiB | 16.50 MiB/s, done.
Resolving deltas: 100% (24/24), done.
/kaggle/working/edm2


In [3]:
# ============================================================
# 📦 Step 1: Count test images dynamically
# ============================================================
import os

test_dir = "/kaggle/input/celeva-64x64-dataset/celeba64/test"
num_test_imgs = sum(1 for f in os.listdir(test_dir) if f.lower().endswith((".png", ".jpg", ".jpeg")))
print(f"✅ Found {num_test_imgs} test images.")

with open("/kaggle/working/num_test.txt", "w") as f:
    f.write(str(num_test_imgs))


✅ Found 19962 test images.


In [4]:
# ============================================================
# 🧩 Step 2: Patch EDM2 Sampler for Logarithmic Sampling
# ============================================================
import torch, types, sys
sys.path.append("/kaggle/working/edm2")
import generate_images as gen

def logarithmic_t_steps(num_steps=32, sigma_min=0.002, sigma_max=80):
    """Logarithmic noise schedule: σ decreases logarithmically from σ_max to σ_min."""
    log_min = torch.log(torch.tensor(sigma_min))
    log_max = torch.log(torch.tensor(sigma_max))
    t = torch.linspace(0, 1, num_steps, device='cuda')
    sigmas = torch.exp(log_max - (log_max - log_min) * t)
    sigmas = torch.cat([sigmas, torch.zeros_like(sigmas[:1])])  # Append σ_N=0
    return sigmas

def logarithmic_sampler(
    net, noise, labels=None, gnet=None,
    num_steps=32, sigma_min=0.002, sigma_max=80, guidance=1,
    S_churn=0, S_min=0, S_max=float('inf'), S_noise=1,
    dtype=torch.float32, randn_like=torch.randn_like,
):
    """Modified EDM2 sampler using a logarithmic noise schedule."""
    def denoise(x, t):
        Dx = net(x, t, labels).to(dtype)
        if guidance == 1:
            return Dx
        ref_Dx = gnet(x, t, labels).to(dtype)
        return ref_Dx.lerp(Dx, guidance)

    # --- Logarithmic σ schedule ---
    t_steps = logarithmic_t_steps(num_steps, sigma_min, sigma_max)
    x_next = noise.to(dtype) * t_steps[0]

    # --- Main denoising loop ---
    for i, (t_cur, t_next) in enumerate(zip(t_steps[:-1], t_steps[1:])):
        x_cur = x_next
        if S_churn > 0 and S_min <= t_cur <= S_max:
            gamma = min(S_churn / num_steps, torch.sqrt(torch.tensor(2.0)) - 1)
            t_hat = t_cur + gamma * t_cur
            x_hat = x_cur + (t_hat**2 - t_cur**2).sqrt() * S_noise * randn_like(x_cur)
        else:
            t_hat = t_cur
            x_hat = x_cur

        d_cur = (x_hat - denoise(x_hat, t_hat)) / t_hat
        x_next = x_hat + (t_next - t_hat) * d_cur

        if i < num_steps - 1:
            d_prime = (x_next - denoise(x_next, t_next)) / t_next
            x_next = x_hat + (t_next - t_hat) * (0.5 * d_cur + 0.5 * d_prime)

    return x_next

# ✅ Inject the patch so EDM2 uses logarithmic sampling
gen.edm_sampler = logarithmic_sampler
print("✅ Logarithmic sampling schedule successfully injected into EDM2 sampler!")


✅ Logarithmic sampling schedule successfully injected into EDM2 sampler!


In [5]:
# ============================================================
# 🖼️ Step 3: Generate images using selected model (Logarithmic Sampling)
# ============================================================
import os

num = int(open("/kaggle/working/num_test.txt").read().strip())
print(f"✅ Generating {num} images with model: {MODEL_PKL}")

os.system(
    f"python /kaggle/working/edm2/generate_images.py "
    f"--net={MODEL_PKL} "
    f"--outdir=/kaggle/working/generated-test-fid "
    f"--seeds=0-{num-1} "
    f"--batch=32 --steps=32 "
    f"--sigma_min=0.002 --sigma_max=80 --rho=7"
)
print("✅ Generation completed.")


✅ Generating 19962 images with model: /kaggle/input/network-snapshot-0002097-0-100-logarithmic-2/pytorch/default/1/network-snapshot-0002097-0.100_logarithmic_noiseinject.pkl


[rank0]:[W1114 15:22:54.384120870 ProcessGroupNCCL.cpp:4561] [PG ID 0 PG GUID 0 Rank 0]  using GPU 0 to perform barrier as devices used by this process are currently unknown. This can potentially cause a hang if this rank to GPU mapping is incorrect. Specify device_ids in barrier() to force use of a particular device, or call init_process_group() with a device_id.


Loading main network from /kaggle/input/network-snapshot-0002097-0-100-logarithmic-2/pytorch/default/1/network-snapshot-0002097-0.100_logarithmic_noiseinject.pkl ...
Setting up StandardRGBEncoder...
Generating 19962 images...


  0%|          | 0/624 [00:00<?, ?batch/s]/kaggle/working/edm2/generate_images.py:235: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  PIL.Image.fromarray(image, 'RGB').save(os.path.join(image_dir, f'{seed:06d}.png'))
100%|██████████| 624/624 [3:06:45<00:00, 17.96s/batch]


✅ Generation completed.


In [6]:
# ============================================================
# 📊 Step 4: Compute Reference Stats (for FID + FD_DINOv2)
# ============================================================
os.system(
    "python /kaggle/working/edm2/calculate_metrics.py ref "
    "--data=/kaggle/input/celeva-64x64-dataset/celeba64/test "
    "--dest=/kaggle/working/celeba64-ref-test.pkl "
    "--metrics=fid,fd_dinov2 "
    "--batch=64"
)


[rank0]:[W1114 18:30:10.059237868 ProcessGroupNCCL.cpp:4561] [PG ID 0 PG GUID 0 Rank 0]  using GPU 0 to perform barrier as devices used by this process are currently unknown. This can potentially cause a hang if this rank to GPU mapping is incorrect. Specify device_ids in barrier() to force use of a particular device, or call init_process_group() with a device_id.
Downloading: "https://github.com/facebookresearch/dinov2/zipball/main" to /root/.cache/dnnlib/torch_hub/main.zip
Downloading: "https://dl.fbaipublicfiles.com/dinov2/dinov2_vitl14/dinov2_vitl14_pretrain.pth" to /root/.cache/dnnlib/torch_hub/checkpoints/dinov2_vitl14_pretrain.pth
  0%|          | 0.00/1.13G [00:00<?, ?B/s]

Loading images from /kaggle/input/celeva-64x64-dataset/celeba64/test ...
Setting up InceptionV3Detector...
Setting up DINOv2Detector...


100%|██████████| 1.13G/1.13G [00:03<00:00, 365MB/s]


Calculating feature statistics...


100%|██████████| 312/312 [18:52<00:00,  3.63s/batch]


0

In [7]:
# ============================================================
# ⚙️ Step 5: Patch FID script to match sample count
# ============================================================
num = int(open("/kaggle/working/num_test.txt").read().strip())
os.system(f'sed -i "s/default=50000/default={num}/" /kaggle/working/edm2/calculate_metrics.py')
print(f"✅ Patched FID script to expect {num} images.")


✅ Patched FID script to expect 19962 images.


In [8]:
# ============================================================
# 📈 Step 6: Compute both FID and FD_DINOv2
# ============================================================
import re, subprocess

num = int(open("/kaggle/working/num_test.txt").read().strip())
cmd = (
    f"python /kaggle/working/edm2/calculate_metrics.py calc "
    f"--images=/kaggle/working/generated-test-fid "
    f"--ref=/kaggle/working/celeba64-ref-test.pkl "
    f"--metrics=fid,fd_dinov2 "
    f"--num={num} "
    f"--batch=64"
)

result = subprocess.getoutput(cmd)
fid_match = re.search(r"fid\s*=\s*([\d.]+)", result)
fd_dino_match = re.search(r"fd_dinov2\s*=\s*([\d.]+)", result)

with open("/kaggle/working/fid_fd_results.txt", "w") as f:
    if fid_match:
        f.write(f"FID = {fid_match.group(1)}\n")
    if fd_dino_match:
        f.write(f"FD_DINOv2 = {fd_dino_match.group(1)}\n")

print(result)
print("✅ Metrics saved to fid_fd_results.txt")


[rank0]:[W1114 18:49:19.031565622 ProcessGroupNCCL.cpp:4561] [PG ID 0 PG GUID 0 Rank 0]  using GPU 0 to perform barrier as devices used by this process are currently unknown. This can potentially cause a hang if this rank to GPU mapping is incorrect. Specify device_ids in barrier() to force use of a particular device, or call init_process_group() with a device_id.
Loading feature statistics from /kaggle/working/celeba64-ref-test.pkl ...
Loading images from /kaggle/working/generated-test-fid ...
Setting up InceptionV3Detector...
Setting up DINOv2Detector...
Calculating feature statistics...

100%|██████████| 312/312 [18:55<00:00,  3.64s/batch]
Calculating fid...
fid = 10.7609
Calculating fd_dinov2...
fd_dinov2 = 451.182
✅ Metrics saved to fid_fd_results.txt
